# Reshaping data for model
### In this file, we will take the data that has been gathered and harmonized and we will fit it, create scenarios, and save it as a structured array for the model. Since we would like to keep the flexibility with excel, we will also save it in an ODYM compatible format and create a file that can do the reverse: if teh excel file is eddited, so is the array. This will be a separate script

In [1]:
# Load a local copy of the current ODYM branch:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import xlrd
import pylab
from copy import deepcopy
import logging as log
import xlwt
import tqdm
import math
from scipy.stats import norm
from openpyxl import *
from scipy.optimize import curve_fit
from scipy.stats import gompertz
from logistic import logistic as logistic
### Preamble
os.getcwd()
os.chdir("..")
os.chdir("..")

In [4]:
xlrd.xlsx.Element_has_iter = True

# add ODYM module directory to system path, relative
MainPath = os.path.join(os.getcwd(), 'odym', 'modules')
sys.path.insert(0, MainPath)

# add ODYM module directory to system path, absolute
sys.path.insert(0, os.path.join(os.getcwd(), 'odym', 'modules'))

# Specify path to dynamic stock model and to datafile, relative
DataPath = os.path.join( 'docs', 'files')

# Specify path to dynamic stock model and to datafile, absolute
DataPath = os.path.join(os.getcwd(), 'docs', 'Files')

import ODYM_Classes as msc # import the ODYM class file
import ODYM_Functions as msf # import the ODYM function file
import dynamic_stock_model as dsm # import the dynamic stock model library

# Initialize loggin routine
log_verbosity = eval("log.DEBUG")
log_filename = 'LogFileTest.md'
[Mylog, console_log, file_log] = msf.function_logger(log_filename, os.getcwd(),
                                                     log_verbosity, log_verbosity)
Mylog.info('### 1. - Initialize.')

#Read main script parameters
#Load project-specific config file
ProjectSpecs_ConFile = 'ODYM_Config_Vehicle_System.xlsx'
Model_Configfile     = xlrd.open_workbook(os.path.join(DataPath, ProjectSpecs_ConFile))
ScriptConfig         = {'Model Setting': Model_Configfile.sheet_by_name('Config').cell_value(3,3)} # Dictionary with config parameters
Model_Configsheet    = Model_Configfile.sheet_by_name('Setting_' + ScriptConfig['Model Setting'])

Name_Scenario        = Model_Configsheet.cell_value(3,3)
print(Name_Scenario)

#Read control and selection parameters into dictionary
ScriptConfig         = msf.ParseModelControl(Model_Configsheet,ScriptConfig)

Mylog.info('Read and parse config table, including the model index table, from model config sheet.')
IT_Aspects,IT_Description,IT_Dimension,IT_Classification,IT_Selector,IT_IndexLetter,\
PL_Names,PL_Description,PL_Version,PL_IndexStructure,PL_IndexMatch,PL_IndexLayer,\
PrL_Number,PrL_Name,PrL_Comment,PrL_Type,ScriptConfig = msf.ParseConfigFile(Model_Configsheet,ScriptConfig,Mylog)    

class_filename       = 'ODYM_Classifications_Master_Vehicle_System.xlsx'
Classfile            = xlrd.open_workbook(os.path.join(DataPath,class_filename))
Classsheet           = Classfile.sheet_by_name('MAIN_Table')
MasterClassification = msf.ParseClassificationFile_Main(Classsheet,Mylog)

Mylog.info('Define model classifications and select items for model classifications according to information provided by config file.')
ModelClassification  = {} # Dict of model classifications
for m in range(0,len(IT_Aspects)):
    ModelClassification[IT_Aspects[m]] = deepcopy(MasterClassification[IT_Classification[m]])
    EvalString = msf.EvalItemSelectString(IT_Selector[m],len(ModelClassification[IT_Aspects[m]].Items))
    if EvalString.find(':') > -1: # range of items is taken
        RangeStart = int(EvalString[0:EvalString.find(':')])
        RangeStop  = int(EvalString[EvalString.find(':')+1::])
        ModelClassification[IT_Aspects[m]].Items = ModelClassification[IT_Aspects[m]].Items[RangeStart:RangeStop]           
    elif EvalString.find('[') > -1: # selected items are taken
        ModelClassification[IT_Aspects[m]].Items = [ModelClassification[IT_Aspects[m]].Items[i] for i in eval(EvalString)]
    elif EvalString == 'all':
        None
    else:
        Mylog.error('Item select error for aspect ' + IT_Aspects[m] + ' were found in datafile.')
        break

# Define model index table and parameter dictionary
Mylog.info('### 2.2 - Define model index table and parameter dictionary')
Model_Time_Start = int(min(ModelClassification['Time'].Items))
Model_Time_End   = int(max(ModelClassification['Time'].Items))
Model_Duration   = Model_Time_End - Model_Time_Start + 1

Mylog.info('Define index table dataframe.')
IndexTable = pd.DataFrame({'Aspect'        : IT_Aspects,  # 'Time' and 'Element' must be present!
                           'Description'   : IT_Description,
                           'Dimension'     : IT_Dimension,
                           'Classification': [ModelClassification[Aspect] for Aspect in IT_Aspects],
                           'IndexLetter'   : IT_IndexLetter})  # Unique one letter (upper or lower case) indices to be used later for calculations.

# Default indexing of IndexTable, other indices are produced on the fly
IndexTable.set_index('Aspect', inplace=True)

# Add indexSize to IndexTable:
IndexTable['IndexSize'] = pd.Series([len(IndexTable.Classification[i].Items) for i in range(0, len(IndexTable.IndexLetter))],
                                    index=IndexTable.index)

# list of the classifications used for each indexletter
IndexTable_ClassificationNames = [IndexTable.Classification[i].Name for i in range(0, len(IndexTable.IndexLetter))]

# Define dimension sizes
Nt = len(IndexTable.Classification[IndexTable.index.get_loc('Time')].Items)
Nc = len(IndexTable.Classification[IndexTable.index.get_loc('Age-cohort')].Items)
Ng = len(IndexTable.Classification[IndexTable.index.get_loc('Good')].Items)
Nr = len(IndexTable.Classification[IndexTable.index.get_loc('Region')].Items)
Ne = len(IndexTable.Classification[IndexTable.index.get_loc('Element')].Items)
Nb = len(IndexTable.Classification[IndexTable.index.get_loc('Battery_Chemistry')].Items)
# Nk = len(IndexTable.Classification[IndexTable.index.get_loc('Capacity')].Items)
Np = len(IndexTable.Classification[IndexTable.index.get_loc('Battery_Parts')].Items)
Ns = len(IndexTable.Classification[IndexTable.index.get_loc('Size')].Items)
Nh = len(IndexTable.Classification[IndexTable.index.get_loc('Recycling_Process')].Items)
# Nv = len(IndexTable.Classification[IndexTable.index.get_loc('Make')].Items)
NS = len(IndexTable.Classification[IndexTable.index.get_loc('Scenario')].Items)

INFO (1863578490.py <<module>>): ### 1. - Initialize.
INFO (1863578490.py <<module>>): Read and parse config table, including the model index table, from model config sheet.
INFO (ODYM_Functions.py <ParseConfigFile>): Read parameter list from model config sheet.
INFO (ODYM_Functions.py <ParseConfigFile>): Read process list from model config sheet.
INFO (ODYM_Functions.py <ParseConfigFile>): Read model run control from model config sheet.
INFO (ODYM_Functions.py <ParseConfigFile>): Read model output control from model config sheet.
INFO (ODYM_Functions.py <ParseClassificationFile_Main>): End of file or formatting error while reading the classification file in column 20. Check if all classifications are present. If yes, you are good to go!
INFO (1863578490.py <<module>>): Define model classifications and select items for model classifications according to information provided by config file.
INFO (1863578490.py <<module>>): ### 2.2 - Define model index table and parameter dictionary


Vehicle stock model for Global fleet


INFO (1863578490.py <<module>>): Define index table dataframe.


## Preparing battery materials per kg

In [5]:
df = pd.read_excel('/Users/fernaag/Library/CloudStorage/Box-Box/BATMAN/Data/Database/data/01_raw_data/Xi_et_al_model_2020/Material_content_new.xlsx', sheet_name='material content per part').drop('Comment', axis=1)

In [6]:
df.tail(20)

,drive_train,part,material,LFP,NCA,NCM111,NCM523,NCM622,NCM622-Graphite (Si),NCM811-Graphite (Si),NCM955-Graphite (Si),Li-Sulphur,Li-Air
14,PHEV,Modules,Ni,0.000000,0.144870,0.065977,0.096778,0.110010,0.117490,0.144693,0.158692,0.000000,0.112708
15,PHEV,Modules,Co,0.000000,0.027265,0.066224,0.038856,0.036807,0.039310,0.018154,0.008849,0.000000,0.000000
16,PHEV,Modules,Mn,0.000000,0.000000,0.061740,0.054338,0.034315,0.036648,0.016925,0.008250,0.000000,0.000000
17,PHEV,Modules,Al,0.414135,0.361206,0.356501,0.359598,0.357957,0.363833,0.355006,0.356173,0.409538,0.447951
18,PHEV,Modules,Cu,0.153812,0.114820,0.117647,0.116871,0.117101,0.123136,0.130036,0.131192,0.475007,0.337997
19,PHEV,Modules,Steel,0.012656,0.011295,0.010979,0.010932,0.011118,0.011221,0.011719,0.011671,0.016525,0.017954
20,PHEV,Modules,Graphite,0.171606,0.193733,0.168129,0.171047,0.182316,0.150951,0.165334,0.167626,0.000000,0.000000
21,PHEV,Modules,Electrolyte,0.147017,0.117668,0.118196,0.117728,0.118191,0.115074,0.116537,0.116303,0.000000,0.000000
22,PHEV,Modules,Separator,0.012220,0.006915,0.007894,0.007741,0.007403,0.008005,0.008449,0.008571,0.000000,0.000000
23,PHEV,Modules,Si,0.000000,0.000000,0.000000,0.000000,0.000000,0.007945,0.008702,0.008822,0.000000,0.000000


In [7]:
df = df.melt(['drive_train','part', 'material'])

In [8]:
df.tail(20)

,drive_train,part,material,variable,value
320,PHEV,Modules,Ni,Li-Air,0.112708
321,PHEV,Modules,Co,Li-Air,0.000000
322,PHEV,Modules,Mn,Li-Air,0.000000
323,PHEV,Modules,Al,Li-Air,0.447951
324,PHEV,Modules,Cu,Li-Air,0.337997
325,PHEV,Modules,Steel,Li-Air,0.017954
326,PHEV,Modules,Graphite,Li-Air,0.000000
327,PHEV,Modules,Electrolyte,Li-Air,0.000000
328,PHEV,Modules,Separator,Li-Air,0.000000
329,PHEV,Modules,Si,Li-Air,0.000000


In [9]:
MatArray = np.zeros((Ng,Nb,Np,Ne))

In [10]:
for m in range(0, len(df.material.values)):
    try:
        DTPosition = IndexTable.Classification[IndexTable.index.get_loc('Good')].Items.index(df.drive_train.iloc[m])
        ChemistryPosition= IndexTable.Classification[IndexTable.index.get_loc('Battery_Chemistry')].Items.index(df.variable.iloc[m])
        PartPosition= IndexTable.Classification[IndexTable.index.get_loc('Battery_Parts')].Items.index(df.part.iloc[m])
        ElementPosition= IndexTable.Classification[IndexTable.index.get_loc('Element')].Items.index(df.material.iloc[m])
        MatArray[DTPosition, ChemistryPosition, PartPosition, ElementPosition] = df.value.iloc[m]
    except ValueError: # This is just to ignore parts that are not included in the model
        pass

In [11]:
MatArray[1,-1,1,:]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [13]:
np.save('/Users/fernaag/Library/CloudStorage/Box-Box/BATMAN/Data/Database/data/03_scenario_data/global_model/materialContent/matContent_motorEnergy_vehicleSize_batteryChemistry', MatArray)

In [15]:
g,b,p,e = pd.core.reshape.util.cartesian_product(
    [
        IndexTable.Classification[IndexTable.index.get_loc("Good")].Items,
        IndexTable.Classification[IndexTable.index.get_loc("Battery_Chemistry")].Items,
        IndexTable.Classification[IndexTable.index.get_loc("Battery_Parts")].Items,
        IndexTable.Classification[IndexTable.index.get_loc("Element")].Items
    ]
    )
    
file = pd.DataFrame(
    dict(Drive_Train=g, Battery_Chemistry=b, Battery_Part=p, Material=e)
)

values = []
for g in range(Ng):
    for b in range(Nb):
        for p in range(Np):
            for e in range(Ne):
                value = MatArray[g,b,p,e]
                values.append(value)

file['value'] = values
file.to_excel('/Users/fernaag/Library/CloudStorage/Box-Box/BATMAN/Data/Database/data/02_harmonized_data/parameter_values/material_content_data.xlsx')
